# E2E scenario for Wine dataset on KFP

Steps:
- download dataset
- clean/preprocess data
- perform training / hyper-parameter tuning with results in MLFlow + MinIO
- serve with Seldon
- perform inference

Artifacts:
- raw data, preprocessed
- model per experiment
- experiment metadata and results

## Tested with

This notebook has been tested with the following core component versions:

|                              |     **Charm**     | **Client** |                            **Image**                           |
|:----------------------------:|:-----------------:|:----------:|:--------------------------------------------------------------:|
| **Kubeflow Pipelines (KFP)** |      2.0/edge     |   1.8.22   |           gcr.io/ml-pipeline/api-server:2.0.0-alpha.7          |
|          **MLFlow**          | latest/edge (2.1) |    2.1.1   |             docker.io/ubuntu/mlflow:2.1.1_1.0-22.04            |
|           **MinIO**          |    ckf-1.7/edge   |    6.0.2   |            minio/minio:RELEASE.2021-09-03T03-56-13Z            |
|          **Seldon**          |     1.15/edge     |     N/A    | docker.io/charmedkubeflow/seldon-core-operator:v1.15.0_22.04_1 |

## Setup

In [ ]:
!pip install boto3 kfp==1.8.22 minio mlflow==2.1.1 numpy pyarrow requests "scikit-learn<1.2" tenacity -q

### Import required packages

In [ ]:
import os

from urllib import request

import kfp
import mlflow
import requests

from kfp import dsl
from kfp.onprem import use_k8s_secret
from kubernetes import client as k8s_client, config as k8s_config
from kubernetes.client.exceptions import ApiException
from kubernetes.client.models import V1EnvVar
from minio import Minio
from minio.error import BucketAlreadyOwnedByYou
from sklearn.linear_model import ElasticNet
from tenacity import retry, stop_after_attempt, wait_exponential

### Initialise KFP Client

In [ ]:
client = kfp.Client()

### Create MinIO Bucket for MLFlow

Create a MinIO bucket for MLFlow if it doesn't already exist.

In [ ]:
MINIO_BUCKET = "mlflow"
MINIO_HOST = os.getenv("MINIO_ENDPOINT_URL").split("http://")[1]

In [ ]:
# initialise MinIO client
mc = Minio(
    endpoint=MINIO_HOST,
    access_key=os.environ["AWS_ACCESS_KEY_ID"],
    secret_key=os.environ["AWS_SECRET_ACCESS_KEY"],
    secure=False,
)

In [ ]:
try:
    mc.make_bucket(MINIO_BUCKET)
except BucketAlreadyOwnedByYou:
    print(f"Bucket {MINIO_BUCKET} already exists!")

## Download Data

In [ ]:
DATA_URL = "https://raw.githubusercontent.com/canonical/kubeflow-examples/main/e2e-wine-kfp-mlflow/winequality-red.csv"
DATA_FILE = "winequality-red.csv"

In [ ]:
# local development
request.urlretrieve(DATA_URL, DATA_FILE)

print(f"File '{DATA_FILE}' downloaded successfully.")

In [ ]:
# workflow component
web_downloader_op = kfp.components.load_component_from_url(
    "https://raw.githubusercontent.com/kubeflow/pipelines/1.8.22/components/contrib/web/Download/component.yaml"
)

## Preprocess Data

In [ ]:
def preprocess(
    file_path: kfp.components.InputPath("CSV"),
    output_file: kfp.components.OutputPath("parquet")
):
    import pandas as pd
    df = pd.read_csv(file_path, header=0, sep=";")
    df.columns = [c.lower().replace(" ", "_") for c in df.columns]
    df.to_parquet(output_file)

In [ ]:
# local development
OUTPUT_PARQUET_FILE = "preprocessed.parquet"
preprocess(DATA_FILE, OUTPUT_PARQUET_FILE)

In [ ]:
assert os.path.exists(OUTPUT_PARQUET_FILE)

In [ ]:
# workflow component
preprocess_op = kfp.components.create_component_from_func(
    func=preprocess,
    output_component_file="preprocess-component.yaml",
    base_image="python:3.8.10",
    packages_to_install=["pandas", "pyarrow"],
)

## Train Model

In [ ]:
def training(file_path: kfp.components.InputPath("parquet")) -> str:
    import os
    import mlflow
    import pandas as pd

    from sklearn.linear_model import ElasticNet
    from sklearn.metrics import classification_report
    from sklearn.model_selection import train_test_split
    
    df = pd.read_parquet(file_path)
    
    target_column="quality"
    train_x, test_x, train_y, test_y = train_test_split(
        df.drop(columns=[target_column]),
        df[target_column], test_size=.25,
        random_state=42, stratify=df[target_column]
    )

    mlflow.sklearn.autolog()
    with mlflow.start_run(run_name="elastic_net_models") as run:
        mlflow.set_tag("author", "kf-testing")
        lr = ElasticNet(alpha=0.5, l1_ratio=0.5, random_state=42)
        lr.fit(train_x, train_y)
        model_dir = "model"
        mlflow.sklearn.log_model(lr, model_dir, registered_model_name="wine-elasticnet")
        return f"{run.info.artifact_uri}/{model_dir}"

In [ ]:
# local development
training(OUTPUT_PARQUET_FILE)

In [ ]:
run = mlflow.last_active_run()

In [ ]:
@retry(
    wait=wait_exponential(multiplier=2, min=1, max=10),
    stop=stop_after_attempt(30),
    reraise=True,
)
def assert_mlflow_run_finished(client, run_id):
    """Wait for the run to complete successfully."""
    status = client.get_run(run_id).info.status
    assert status == "FINISHED", f"MLFlow run in {status} state."


def assert_mlflow_model_exists(client, run_id):
    """Assert Model exists."""
    model = client.sklearn.load_model(f"runs:/{run_id}/model")
    assert isinstance(model, ElasticNet), f"Model {model} is not of type ElasticNet!"

In [ ]:
assert run.data.tags.get("author") == "kf-testing"
assert_mlflow_run_finished(mlflow, run.info.run_id)
assert_mlflow_model_exists(mlflow, run.info.run_id)

In [ ]:
# workflow component
training_op = kfp.components.create_component_from_func(
    func=training,
    output_component_file="train-component.yaml",
    base_image="python:3.8.10",
    packages_to_install=["boto3", "mlflow==2.1.1", "numpy", "pandas", "pyarrow", "scikit-learn<1.2"],
)

## Deploy Model

In [ ]:
SELDON_DEPLOYMENT_NAME = "kf-testing"
SELDON_IMAGE = "seldonio/mlflowserver:1.17.0"
MODEL_NAME = "wine-model"

In [ ]:
def deploy(
    seldon_deployment_name: str = "default_seldon_deployment_name",
    seldon_image: str = "default_seldon_image",
    model_uri: str = "default_model_uri",
    model_name: str = "default_model_name",
):
    import yaml

    from kubernetes import client, config

    manifest = """
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: """ + seldon_deployment_name + """
spec:
  name: wines
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - name: classifier
          image: """ + seldon_image + """
          imagePullPolicy: Always
          livenessProbe:
            initialDelaySeconds: 80
            failureThreshold: 200
            periodSeconds: 5
            successThreshold: 1
            httpGet:
              path: /health/ping
              port: http
              scheme: HTTP
          readinessProbe:
            initialDelaySeconds: 80
            failureThreshold: 200
            periodSeconds: 5
            successThreshold: 1
            httpGet:
              path: /health/ping
              port: http
              scheme: HTTP
    graph:
      children: []
      implementation: MLFLOW_SERVER
      modelUri: """ + model_uri + """
      envSecretRefName: mlflow-server-seldon-rclone-secret
      name: classifier
    name: """ + model_name + """
    replicas: 1
    """

    with open("/var/run/secrets/kubernetes.io/serviceaccount/namespace", "r") as f:
        namespace = f.read().strip()

    config.load_incluster_config()
    api_instance = client.ApiClient()
    custom_api = client.CustomObjectsApi(api_instance)

    try:
        api_response = custom_api.create_namespaced_custom_object(
            group="machinelearning.seldon.io",
            version="v1",
            plural="seldondeployments",
            namespace=namespace,
            body=yaml.safe_load(manifest),
        )
        print("Custom Resource applied successfully.")
        print(api_response)
    except client.rest.ApiException as e:
        print(f"Failed to apply Custom Resource: {e}")

In [ ]:
# workflow component
deploy_op = kfp.components.create_component_from_func(
    func=deploy,
    output_component_file="deploy-component.yaml",
    base_image="python:3.8.10",
    packages_to_install=["kubernetes", "pyyaml"],
)

## Create Pipeline

In [ ]:
@dsl.pipeline(
    name="e2e_wine_pipeline",
    description="E2E Wine Pipeline with MLFlow and Seldon",
)
def wine_pipeline(url, seldon_deployment_name, seldon_image, model_name):
    web_downloader_task = web_downloader_op(url=url)
    preprocess_task = preprocess_op(file=web_downloader_task.outputs["data"])
    train_task = (
        training_op(file=preprocess_task.outputs["output"])
        .add_env_variable(V1EnvVar(name="MLFLOW_TRACKING_URI", value=os.getenv("MLFLOW_TRACKING_URI")))
        .add_env_variable(V1EnvVar(name="MLFLOW_S3_ENDPOINT_URL", value=os.getenv("MLFLOW_S3_ENDPOINT_URL")))
        .add_env_variable(V1EnvVar(name="PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION", value="python"))
        .apply(
            use_k8s_secret(
                secret_name="mlflow-server-minio-artifact",
                k8s_secret_key_to_env={
                    "AWS_ACCESS_KEY_ID": "AWS_ACCESS_KEY_ID",
                    "AWS_SECRET_ACCESS_KEY": "AWS_SECRET_ACCESS_KEY",
                },
            )
        )
    )
    deploy_task = deploy_op(
        seldon_deployment_name=seldon_deployment_name,
        seldon_image=seldon_image,
        model_uri=train_task.output,
        model_name=model_name,
    )

In [ ]:
# local development
kfp.compiler.Compiler().compile(wine_pipeline, "wine-pipeline.yaml")

In [ ]:
run = client.create_run_from_pipeline_func(
    wine_pipeline,
    arguments={
        "url": DATA_URL,
        "seldon_deployment_name": SELDON_DEPLOYMENT_NAME,
        "seldon_image": SELDON_IMAGE,
        "model_name": MODEL_NAME,
    },
)

## Monitor KFP Run

Wait for the KFP run to be completed successfully.

In [ ]:
@retry(
    wait=wait_exponential(multiplier=2, min=1, max=10),
    stop=stop_after_attempt(30),
    reraise=True,
)
def assert_kfp_run_succeeded(client, run_id):
    """Wait for the run to complete successfully."""
    status = client.get_run(run_id).run.status
    assert status == "Succeeded", f"KFP run in {status} state."

In [ ]:
assert_kfp_run_succeeded(client, run.run_id)

## Perform Inference

Wait for the SeldonDeployment to become available and hit it for predictions.

### Setup K8s Client

In [ ]:
with open("/var/run/secrets/kubernetes.io/serviceaccount/namespace", "r") as f:
    NAMESPACE = f.read().strip()

k8s_config.load_incluster_config()
api_instance = k8s_client.ApiClient()
custom_api = k8s_client.CustomObjectsApi(api_instance)

### Define K8s Helpers

In [ ]:
def get_seldon_deployment(name, namespace):
    """Get SeldonDeployment by name."""
    return custom_api.get_namespaced_custom_object(
        group="machinelearning.seldon.io",
        version="v1",
        plural="seldondeployments",
        namespace=namespace,
        name=name,
    )

def delete_seldon_deployment(name, namespace):
    """Delete SeldonDeployment by name."""
    return custom_api.delete_namespaced_custom_object(
        group="machinelearning.seldon.io",
        version="v1",
        plural="seldondeployments",
        namespace=namespace,
        name=name,
    )

### Wait for SeldonDeployment to Become Available

In [ ]:
@retry(
    wait=wait_exponential(multiplier=2, min=1, max=10),
    stop=stop_after_attempt(50),
    reraise=True,
)
def assert_seldon_deployment_available(seldon_deployment_name, namespace):
    """Wait for the SeldonDeployment to become available."""
    try:
        sd = get_seldon_deployment(seldon_deployment_name, namespace)
    except ApiException as err:
        assert err.status != 404, f"SeldonDeployment {seldon_deployment_name} not found!"
        raise
    status = sd.get("status")
    assert status is not None, "SeldonDeployment status not yet available!"
    assert status.get("state") == "Available"

In [ ]:
assert_seldon_deployment_available(SELDON_DEPLOYMENT_NAME, NAMESPACE)

### Hit SeldonDeployment for Predictions

In [ ]:
sd = get_seldon_deployment(SELDON_DEPLOYMENT_NAME, NAMESPACE)
url = sd['status']['address']['url']
print("SeldonDeployment URL:", url)

In [ ]:
inference_input = {
  "data": {
      "ndarray": [
          [
              10.1, 0.37, 0.34, 2.4, 0.085, 5.0, 17.0, 0.99683, 3.17, 0.65, 10.6
          ]
      ]
  }
}
response = requests.post(url, json=inference_input)
print(response.text)

In [ ]:
res = response.json()
# verify that the predictions are as expected
assert res.get("data") and res.get("data").get("ndarray"), "Failed to get predictions!"
predictions = res["data"]["ndarray"]
assert len(predictions) == 1, "Predictions not in the expected format!"
assert predictions == [5.737135502528464], "Predictions different than expected!"

## Delete SeldonDeployment

In [ ]:
delete_seldon_deployment(SELDON_DEPLOYMENT_NAME, NAMESPACE);

In [ ]:
@retry(
    wait=wait_exponential(multiplier=2, min=1, max=10),
    stop=stop_after_attempt(30),
    reraise=True,
)
def assert_seldon_deployment_deleted(seldon_deployment_name, namespace):
    """Wait for the SeldonDeployment to be deleted."""
    try:
        # try fetching the SeldonDeployment to verify it was deleted successfully
        sd = get_seldon_deployment(seldon_deployment_name, namespace)
        assert not sd, f"Failed to delete SeldonDeployment {seldon_deployment_name}!"
    except ApiException as err:
        assert err.status == 404, f"Caught unexpected exception: {err}"

In [ ]:
assert_seldon_deployment_deleted(SELDON_DEPLOYMENT_NAME, NAMESPACE)